In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210529.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice]
0,25/05/2021 17:40:43,Elevator to the Gallows (1958),Streets of Fire (1984),Purple Rain (1984),NaN,BOTCHED HITS,THE BOYS OF SUMMER,FILM NOIRE,EARLY 2000s,[Default to Nicolas Cage]
1,25/05/2021 18:08:59,Purple Rain (1984),Repo Man (1984),Elevator to the Gallows (1958),Streets of Fire (1984),EARLY 2000s,THE BOYS OF SUMMER,FILM NOIRE,[Default to Nicolas Cage],BOTCHED HITS
2,26/05/2021 05:37:39,Streets of Fire (1984),Repo Man (1984),Purple Rain (1984),Elevator to the Gallows (1958),EARLY 2000s,BOTCHED HITS,THE BOYS OF SUMMER,FILM NOIRE,[Default to Nicolas Cage]
3,26/05/2021 21:57:49,Purple Rain (1984),NaN,NaN,NaN,THE BOYS OF SUMMER,EARLY 2000s,BOTCHED HITS,[Default to Nicolas Cage],FILM NOIRE
4,26/05/2021 22:26:58,Elevator to the Gallows (1958),Streets of Fire (1984),Repo Man (1984),Purple Rain (1984),FILM NOIRE,THE BOYS OF SUMMER,BOTCHED HITS,EARLY 2000s,[Default to Nicolas Cage]
5,27/05/2021 10:32:09,Streets of Fire (1984),Repo Man (1984),Elevator to the Gallows (1958),Purple Rain (1984),EARLY 2000s,THE BOYS OF SUMMER,BOTCHED HITS,FILM NOIRE,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Purple Rain (1984)', 'Repo Man (1984)',
       'Elevator to the Gallows (1958)', 'Streets of Fire (1984)'],
      dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Purple Rain (1984)': <Candidate('Purple Rain (1984)')>,
 'Repo Man (1984)': <Candidate('Repo Man (1984)')>,
 'Elevator to the Gallows (1958)': <Candidate('Elevator to the Gallows (1958)')>,
 'Streets of Fire (1984)': <Candidate('Streets of Fire (1984)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Elevator to the Gallows (1958), Streets of Fire (1984), Purple Rain (1984))>,
 <Ballot(Purple Rain (1984), Repo Man (1984), Elevator to the Gallows (1958), Streets of Fire (1984))>,
 <Ballot(Streets of Fire (1984), Repo Man (1984), Purple Rain (1984), Elevator to the Gallows (1958))>,
 <Ballot(Purple Rain (1984))>,
 <Ballot(Elevator to the Gallows (1958), Streets of Fire (1984), Repo Man (1984), Purple Rain (1984))>,
 <Ballot(Streets of Fire (1984), Repo Man (1984), Elevator to the Gallows (1958), Purple Rain (1984))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                         Votes  Status
------------------------------  -------  --------
Streets of Fire (1984)                2  Hopeful
Purple Rain (1984)                    2  Hopeful
Elevator to the Gallows (1958)        2  Rejected
Repo Man (1984)                       0  Rejected

FINAL RESULT
Candidate                         Votes  Status
------------------------------  -------  --------
Streets of Fire (1984)                4  Elected
Purple Rain (1984)                    2  Rejected
Elevator to the Gallows (1958)        0  Rejected
Repo Man (1984)                       0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                    Votes  Status
-------------------------  -------  --------
EARLY 2000s                      3  Hopeful
BOTCHED HITS                     1  Hopeful
THE BOYS OF SUMMER               1  Rejected
FILM NOIRE                       1  Rejected
[Default to Nicolas Cage]        0  Rejected

FINAL RESULT
Candidate                    Votes  Status
-------------------------  -------  --------
EARLY 2000s                      4  Elected
BOTCHED HITS                     2  Rejected
THE BOYS OF SUMMER               0  Rejected
FILM NOIRE                       0  Rejected
[Default to Nicolas Cage]        0  Rejected

